# Relatório Criação do Algoritmo de Resolução do Wordle:

## Etapa 1: Pegar Lista de Palavras

* Primeiramente necessitamos de uma lista de palavras possíveis para a reposta do nosso problema;
* Para isso utilizamos uma lista de palavras com 5 letras de um site brasileiro onde elas foram tratadas e normalizadas;

* Importamos os pacotes que vamos utilizar:

In [2]:
import pandas as pd
import math

In [3]:
word_df = pd.read_csv('clean_words.csv', sep = ",", header = None)
word_df.head(5)

,0
0,aarao
1,abaca
2,abaci
3,abaco
4,abade


* Para ter uma leitura melhor vamos renomear a coluna 0;

In [4]:
word_df.rename(columns={0: "Palavra"}, inplace=True)
word_df.sample(10)

,Palavra
612,batch
1328,crega
6055,terio
749,bomba
5065,uivem
5181,vazar
2946,mamba
3598,parte
3349,noemi
4939,touco


* Vamos transformar o objeto em uma lista de strings:

In [5]:
word_list = list(word_df['Palavra'])
word_list = tuple(word_list)

## Etapa 2: Criar padrões

* Para resolver o problema precisamos criar os padrões de resposta:
* Vamos usar um padrão que é:
    * Formado por 5 letras:
    * Caso a letra seja 'C' ela não está presente na resposta;
    * Caso a letra seja 'Y' ela está presente mas na está no lugar certo;
    * Caso a letra seja 'G' ela está presente e no lugar certo;

* Exemplo, a tentativa do usuário é 'serao' e a resposta é 'errar':
    * O padrão recebido será: 'YCGGY';

* Agora vamos criar esse algoritmo:
* Primeiramente precisamo criar uma função que preencha o padrão com as letras que estão no lugar certo; 

In [6]:
def find_green(attempt, correct):
    pattern = []
    for index in range(5):
        if attempt[index] == correct[index]:
            pattern.append('G')
        else:
            pattern.append('C')
    
    return ''.join(pattern)

In [7]:
find_green("serao", "errar")

'CCGGC'

* Pronto já temos qual são as letras que estão no lugar certo, mas como faremos para saber quais letras estão no lugar errado;
* Primeiro precisamos pular as que estão no lugar certo, para não sobreescreve-las;
* Outro problema é não marcar uma letra repetida duas vezes;

* Exemplo tentativa é __abada__ e a palavra é __caixa__:
    * O padrão esperado é: __YCCCG__ e não __YCYCG__;
    * No segundo caso, as duas letras a da palavra __caixa__ são contadas como 3 letras;
    * Por isso deletamos a letra dos valores da lista correta assim que achamos uma correspondente na tentativa;

* O código que fiz para preencher os valores amarelos é:

In [8]:
def find_yellow(attempt, correct):
    attempt = list(attempt)
    correct = list(correct)
    
    for index in range(5):
        if attempt[index] in correct and index not in correct_indexes:
            pattern[index] = 'Y'
            index_achado = correct.index(attempt[index])
            del correct[index_achado]


* Vendo o código vemos que precisamos guardar os valores que estão green;
* E passamos desnecessariamente valores de uma função para outra, a saída para isso foi a criação de uma classe:

In [36]:
class CreatePattern:
    def __init__(self, correct, attempt) -> None:
        self.correct = list(correct)
        self.attempt = list(attempt)
        self.pattern = list('CCCCC')
        self.correct_indexes = []

    def definy_pattern(self):
        try:
            self.find_green()
            self.find_yellow()
            return ''.join(self.pattern)
        except Exception:
            print(f'Palavra que deu erro {self.attempt} com {self.correct}')

    def find_green(self):
        for index in range(5):
            if self.attempt[index] == self.correct[index]:
                self.pattern[index] = 'G'
                self.correct_indexes.append(index)
                self.correct[index] = '1'
    
    def find_yellow(self):
        for index in range(5):
            if self.attempt[index] in self.correct and index not in self.correct_indexes:
                index_achado = self.correct.index(self.attempt[index])
                if index_achado not in self.correct_indexes:
                    self.pattern[index] = 'Y'
                del self.correct[index_achado]

In [10]:
CreatePattern('caixa', 'abada').definy_pattern()

'YCCCG'

## Etapa 3: Criando agora uma classe para descobrir a melhor palavra para uma lista

* Primeiro precisamos de uma forma de definir qual eh a melhor palavra:

* Para isso vamos utilizar a entropia média que cada palavra da lista vai nos dar, isto é, o quanto aquela palavra vai diminuir a quantidade de possíveis palavras;
* A entropia é calculada por: E = -log2( 1 / Possibilidades )

* Por motivos de memória e otimização vamos calcular a entropia de uma palavra por vez, e vamos usar como exemplo a palavra: __caixa__;

* Primeiro vamos gerar o padrão recebido para cada palavra correta no caso da tentativa ser __'caixa'__;
* Criaremos um dataFrame com isso;

In [11]:
attempt = 'caixa'

padroes = []

for word in word_list:
    padroes.append(CreatePattern(word, attempt).definy_pattern())

padroes_dt = pd.DataFrame([padroes, word_list])

padroes_dt = padroes_dt.T
padroes_dt.rename(columns = {0: 'Pattern', 1: 'Word'}, inplace = True)

* Precisamos transpor a matriz que recebemos:
* E vamos mudar os nomes das colunas para ficar mais claro;

In [12]:
entropy_df = pd.DataFrame([(padroes_dt.Pattern.value_counts())])
entropy_df.head()

entropy_df = entropy_df.T
entropy_df

,Pattern
CCCCC,1006
CYCCC,665
CCYCC,574
CCCCG,366
CGCCC,361
...,...
GGYYC,1
CCYGC,1
CYCGY,1
GYYCG,1


* Agora vamos renomear a coluna para melhor leitura;

In [13]:
entropy_df.rename(columns={'Pattern': 'Total'}, inplace=True)
entropy_df

,Total
CCCCC,1006
CYCCC,665
CCYCC,574
CCCCG,366
CGCCC,361
...,...
GGYYC,1
CCYGC,1
CYCGY,1
GYYCG,1


* Vamos calcular o total de palavras possíveis para calcular a porcentagem;

In [14]:
total = len(word_list)

* Vamos adicionar agora a coluna de porcentagens de possíveis palavras para cada um dos padrões;

In [15]:
entropy_df['Porcentagem'] = entropy_df['Total'].apply(lambda e: e * 100.0 / total)
entropy_df.head()

,Total,Porcentagem
CCCCC,1006,16.456732
CYCCC,665,10.878456
CCYCC,574,9.389825
CCCCG,366,5.987240
CGCCC,361,5.905447


* Agora vamos calcular a entropia para cada um dos padrões;

In [16]:
entropy_df['Entropia'] = entropy_df['Porcentagem'].apply(lambda percentage: math.log2( 100 / (percentage)))
entropy_df.head()

,Total,Porcentagem,Entropia
CCCCC,1006,16.456732,2.603250
CYCCC,665,10.878456,3.200454
CCYCC,574,9.389825,3.412758
CCCCG,366,5.987240,4.061965
CGCCC,361,5.905447,4.081810


* Agora vamos usar a entropia que recebemos para cada padrão para calcular a entropia média para a palavra;
* Para isso iremos iterar através das linhas:
    * Para cada linha iremos multiplicar a entropia de cada linha pelo total de palavras e adicionaremos ao acumulador;
    * Ao final, dividiremos o acumulador pelo total de palavras para recebermos a entropia média da palavra;

In [17]:
sum = 0

for row in entropy_df.iterrows():
    sum += (row[1].Total * row[1].Entropia)

entropy = sum / total
print(f'A entropia média da palavra {attempt} é {entropy}')

A entropia média da palavra caixa é 4.826021179676948


* Agora vamos criar a Classe para fazer tudo isso por nós:

In [18]:
class EntropyGetter:
    def __init__(self, word ,possible_word_list):
        self.word = word
        self.possible_word_list = possible_word_list
        self.total = len(possible_word_list)
    
    def entropy(self):
        self.get_df()
        self.manipulate_dataframe()
        return self.get_entropy()

    def get_entropy(self):
        sum = 0

        for row in self.entropy_df.iterrows():
            sum += (row[1].Total * row[1].Entropia)

        return float(sum) / self.total
    
    def manipulate_dataframe(self):
        self.manipulate_columns()
        self.add_percentage_to_df()
        self.add_entropy_to_df()
    
    def manipulate_columns(self):
        self.entropy_df = self.entropy_df.T
        self.entropy_df.rename(columns={0: 'Total'}, inplace=True)

    def add_entropy_to_df(self):
        self.entropy_df['Entropia'] = self.entropy_df['Porcentagem'].apply(lambda percentage: math.log2( 100 / (percentage)))

    def add_percentage_to_df(self):
        self.entropy_df['Porcentagem'] = self.entropy_df['Total'].apply(lambda e: e * 100.0 / self.total)

    def get_df(self):
        padroes = []
        for word in self.possible_word_list:
            padroes.append(CreatePattern(word, self.word).definy_pattern())

        self.padroes_series = pd.Series(padroes)
        self.entropy_df = pd.DataFrame([self.padroes_series.value_counts()])
    
    

* Agora vamos testar nossa classe;

In [19]:
print(f'A entropia média da palavra {attempt} é {EntropyGetter("caixa", word_list).entropy()}')

A entropia média da palavra caixa é 4.826021179676948


* Agora já temos nossa classe para pegar a entropia de uma palavra específica para uma lista de palavras possíveis;

## Etapa 4: Receber as possíveis palavras para o padrão recebido:

In [20]:
pattern = 'CCYYC'
possible_word_list = list(word_list)
for word in word_list:
    pattern_class = CreatePattern(word, attempt)
    patter_received = pattern_class.definy_pattern()
    del pattern_class
    if (patter_received != pattern):
        possible_word_list.remove(word)

len(possible_word_list)

31

In [21]:
pattern_received = 'CCYYC'

padroes_dt_filtered = padroes_dt[padroes_dt['Pattern'] == pattern_received]

In [22]:
possible_word_list = list(padroes_dt_filtered.Word)
possible_word_list

['eixos',
 'felix',
 'fixei',
 'fixem',
 'fixes',
 'fixou',
 'index',
 'linux',
 'lixos',
 'luxei',
 'noxio',
 'pirex',
 'puxei',
 'remix',
 'rexio',
 'rilex',
 'silex',
 'sioux',
 'xelim',
 'xenio',
 'ximbe',
 'xinto',
 'xopim',
 'xotei',
 'fenix',
 'minix',
 'mixer',
 'pioix',
 'pixel',
 'sexti',
 'xisto']

* Temos aqui agora a lista de possiveis palavras;

In [23]:
len(possible_word_list)

31

## Etapa 5: Agora vamos pegar a melhor palavra para uma lista de possíveis chutes e uma lista de possíveis palavras;

* Para fazer isso precisamos saber qual a palavra dos possíveis chutes vai nos dar a maior entropia-média para uma certa lista de possíveis palavras;

In [24]:
entropy = []

for word in word_list:
    entropy.append(EntropyGetter(word, possible_word_list).entropy())

entropy_mean_df = pd.DataFrame([entropy, word_list])
entropy_mean_df.head()

,0,1,2,3,4,5,6,7,8,9,...,6103,6104,6105,6106,6107,6108,6109,6110,6111,6112
0,2.018399,0.205593,0.834646,1.524526,1.279038,1.76896,2.153075,0.205593,0.951086,1.895721,...,3.471159,2.964177,3.977917,3.332756,3.079818,3.001637,1.601619,3.275516,1.653543,2.143893
1,aarao,abaca,abaci,abaco,abade,abafe,abafo,abaja,abaju,abale,...,xenon,xerez,xerox,xisto,xnove,xtudo,zambi,zepto,zoico,zumbi


* Depois disso manipulamos o DataFrame;

In [25]:
entropy_mean_df = entropy_mean_df.T
entropy_mean_df.head()

,0,1
0,2.018399,aarao
1,0.205593,abaca
2,0.834646,abaci
3,1.524526,abaco
4,1.279038,abade


In [26]:
entropy_mean_df.rename(columns={0: 'Mean-Entropy', 1: 'Word'}, inplace = True)
entropy_mean_df.head()

,Mean-Entropy,Word
0,2.018399,aarao
1,0.205593,abaca
2,0.834646,abaci
3,1.524526,abaco
4,1.279038,abade


* Por fim recebemos a palavra que vai nos dar a maior entropia de todas;

In [27]:
entropy_mean_df[entropy_mean_df['Mean-Entropy'] == entropy_mean_df['Mean-Entropy'].max()]['Word']

4483    silex
Name: Word, dtype: object

* Agora vamos criar uma classe para isso, juntando a etapa 4 e 5;

In [28]:
class GetBetterGuessForInput:
    def __init__(self, possible_word_list: list , word_list: list, pattern: str, word_input: str):
        self.pattern = pattern
        self.word_list = word_list
        self.possible_word_list = possible_word_list
        self.word_input = word_input

    def get_guess(self):
        self.new_possible_words()
        self.get_mean_entropy_df()
        self.manipulate_df()
        return (self.new_possible_words, self.get_max_entropy_guess())
    
    def new_possible_words(self):
        self.new_possible_words = []
        for word in self.possible_word_list:
            pattern_class = CreatePattern(word, self.word_input)
            patter_received = pattern_class.definy_pattern()
            del pattern_class
            if (patter_received == self.pattern):
                self.new_possible_words.append(word)
    
    def get_mean_entropy_df(self):
        entropy = []

        for word in self.word_list:
            entropy_getter = EntropyGetter(word, self.new_possible_words)
            entropy.append(entropy_getter.entropy())
            del entropy_getter

        self.entropy_mean_df = pd.DataFrame([entropy, self.word_list])

    def manipulate_df(self):
        self.entropy_mean_df = self.entropy_mean_df.T
        self.entropy_mean_df.rename(columns={0: 'Mean-Entropy', 1: 'Word'}, inplace = True)

    def get_max_entropy_guess(self):
        max_mean_entropy = self.entropy_mean_df['Mean-Entropy'].max()
        target_row = self.entropy_mean_df[self.entropy_mean_df['Mean-Entropy'] == max_mean_entropy]
        return target_row['Word'].values[0]


* Vamos testar tudo de uma vez agora:

In [29]:
GetBetterGuessForInput(list(word_list), list(word_list), pattern_received, attempt).get_guess()

(['eixos',
  'felix',
  'fixei',
  'fixem',
  'fixes',
  'fixou',
  'index',
  'linux',
  'lixos',
  'luxei',
  'noxio',
  'pirex',
  'puxei',
  'remix',
  'rexio',
  'rilex',
  'silex',
  'sioux',
  'xelim',
  'xenio',
  'ximbe',
  'xinto',
  'xopim',
  'xotei',
  'fenix',
  'minix',
  'mixer',
  'pioix',
  'pixel',
  'sexti',
  'xisto'],
 4483    silex
 Name: Word, dtype: object)

* Pronto dessa forma, agora está pronto nosso algoritmo e podemos usar ele da forma que desejarmos;

* Por questões de maior legibilidade vamos separar em duas classes:
    1. Vamos conseguir a lista de possíveis palavras;
    2. Depois vamos pegar o melhor chute;

In [54]:
class GetPossibleWordList:
    def __init__(self, possible_word_list: list , pattern: str, word_input: str):
        self.pattern = pattern
        self.possible_word_list = possible_word_list
        self.word_input = word_input
    
    def get_new_word_list(self):
        self.fill_new_possible_words()
        return self.new_possible_words

    def fill_new_possible_words(self):
        self.new_possible_words = []
        for word in self.possible_word_list:
            self.check_if_word_is_possible(word)
    
    def check_if_word_is_possible(self, word):
        pattern_class = CreatePattern(word, self.word_input)
        patter_received = pattern_class.definy_pattern()
        del pattern_class
        if (patter_received == self.pattern):
            self.new_possible_words.append(word)

In [58]:
new_word_list = GetPossibleWordList(list(word_list), pattern_received, attempt).get_new_word_list()

In [59]:
class GetBetterGuessForWordList:
    def __init__(self, possible_word_list: list , word_list: list):
        self.word_list = word_list
        self.possible_word_list = possible_word_list
    
    def get_guess(self):
        self.get_mean_entropy_df()
        self.manipulate_df()
        return self.get_max_entropy_guess()
        
    def get_mean_entropy_df(self):
        entropy = []

        for word in self.word_list:
            entropy_getter = EntropyGetter(word, self.possible_word_list)
            entropy.append(entropy_getter.entropy())
            del entropy_getter

        self.entropy_mean_df = pd.DataFrame([entropy, self.word_list])
    
    def manipulate_df(self):
        self.entropy_mean_df = self.entropy_mean_df.T
        self.entropy_mean_df.rename(columns={0: 'Mean-Entropy', 1: 'Word'}, inplace = True)

    def get_max_entropy_guess(self):
        max_mean_entropy = self.entropy_mean_df['Mean-Entropy'].max()
        target_row = self.entropy_mean_df[self.entropy_mean_df['Mean-Entropy'] == max_mean_entropy]
        return target_row['Word'].values[0]

In [60]:
GetBetterGuessForWordList(new_word_list, list(word_list)).get_guess()

'silex'

* Pronto agora já temos as classes separadas, e funcionando;